In [31]:
import glob
from os import path
import json
import numpy as np
from scipy.stats import sem
import pandas as pd

In [32]:
abl_fp = "/data/b2p-siteident/experiments/new_baseline_rgb"

In [37]:
header = [
    "version", "freeze", "tile_size"
]
metrics = [
    # "test_weighted_f1",
    # "test_balanced_acc",
    # "test_rwanda_weighted_f1",
    # "test_rwanda_balanced_acc",
    # "test_uganda_weighted_f1",
    # "test_uganda_balanced_acc",
    "test_balanced_acc", 
    # "test_weighted_f1",
    "ecount"
]

entries_dict = {}
for exp_fp in sorted(glob.glob(path.join(abl_fp, "*"))):
    if not path.isfile(path.join(exp_fp, "test_stats.json")):
            print("{} not finished.".format(exp_fp))
            continue
    with open(path.join(exp_fp, "opts.json")) as f:
        opts = json.load(f)
    
    with open(path.join(exp_fp, "test_stats.json")) as f:
        stats = json.load(f)
    
    data_version = opts["dataset_name"].split("_")[0]
    freeze = opts["use_last_n_layers"]
    tile_size = opts["dataset_name"].split("_")[-2]

    key = (data_version, freeze, tile_size)
    if key not in entries_dict:
        entries_dict[key] = {m: [] for m in metrics}
        entries_dict[key]["ecount"] = 0
        entries_dict[key]["path"] = exp_fp
    for m in metrics:
        if m == "ecount":
            continue
        entries_dict[key][m].append(stats[m])
    entries_dict[key]["ecount"] += 1

entries = []
i = 0
for k, entry in entries_dict.items():
    avg_entry = list(k)
    for m in metrics:
        if m == "ecount":
            avg_entry.append(entry[m])
        else:
            val = np.array(entry[m])
            if "acc" in m:
                val *= 100
            avg_entry.append(round(np.mean(val), 2))
            avg_entry.append(round(sem(val), 2))
    entries.append(avg_entry)
    i += 1

metrics_header = []
for m in metrics:
    if m == "ecount":
        metrics_header.append(m)
    else:
        metrics_header.append(m + "_m")
        metrics_header.append(m + "_ste")
df = pd.DataFrame(entries, columns=header + metrics_header)

In [38]:
df[df.ecount == 3].sort_values(by=["tile_size", "freeze", "version"])

,version,freeze,tile_size,test_balanced_acc_m,test_balanced_acc_ste,ecount
0,v1,-1,1200,57.69,0.75,3
2,v2,-1,1200,48.85,0.81,3
1,v1,9,1200,56.10,0.77,3
3,v2,9,1200,54.26,0.70,3
4,v1,-1,320,56.56,0.94,3
6,v2,-1,320,51.69,2.00,3
5,v1,9,320,52.74,0.77,3
7,v2,9,320,50.85,0.14,3
8,v1,-1,600,55.35,0.47,3
10,v2,-1,600,46.35,0.71,3


\res{49.06}{0.90} & \res{66.53}{1.77} & & \res{49.10}{0.23} & \res{84.11}{4.97}